In [1]:
# create a hdf5 dataset out of goetiff files

In [1]:
import numpy as np
import pandas as pd
import h5py

from osgeo import gdal
from osgeo import gdal_array
from osgeo import osr

from glob import glob
#from os import listdir
#from os.path import splitext
import logging
import os

In [2]:
# directory paths
#dir_img = '/media/philipp/ed7d22ba-5a3b-4d31-bf6c-6add6e106b3d/test/256x256/1m/data_256_ortho/'
#dir_mask = '/media/philipp/ed7d22ba-5a3b-4d31-bf6c-6add6e106b3d/test/256x256/1m/data_256_gt_roads/'
#dir_img = '/home/philipp/Data/edin_dataset/'
dir_img = '/mnt/sda1/tiles_2018_wien/'

In [3]:
19636 + 19309

38945

In [4]:
# create 
sufix = 'ortho/'
ids_ortho = [os.path.splitext(file)[0] for file in os.listdir(dir_img+sufix) if not file.startswith('.')]
#sufix = 'dsm/'
#ids_dsm = [os.path.splitext(file)[0] for file in os.listdir(dir_img+sufix) if not file.startswith('.')]
#sufix = 'dtm/'
#ids_dtm = [os.path.splitext(file)[0] for file in os.listdir(dir_img+sufix) if not file.startswith('.')]
#sufix = 'slope/'
#ids_slope = [os.path.splitext(file)[0] for file in os.listdir(dir_img+sufix) if not file.startswith('.')]
sufix = 'ground_truth/'
ids_ground_truth = [os.path.splitext(file)[0] for file in os.listdir(dir_img+sufix) if not file.startswith('.')]


## finding files

In [5]:

ids_ortho = []
ids_ground_truth = []

sufix_ortho = 'ortho/'
sufix_gt = 'ground_truth/'

# loop over all files found in directory
for file in os.listdir(dir_img+sufix_ortho):
    # create path to ground truth
    path_gt = dir_img + sufix_gt + 'tile_ground_truth' + file[file.rfind('_'):]
    # if file is .tif and the ID is alo found in the grooound truth dictonary
    if file[file.rfind('.'):] == '.tif' and os.path.isfile(path_gt):
        # add path to lists
        ids_ortho.append(dir_img + sufix_ortho + file)
        ids_ground_truth.append(path_gt)

In [6]:
print(len(ids_ortho))
print(len(ids_ground_truth))

19303
19303


In [7]:
print(ids_ortho[:2])
'''
print(ids_dsm[:2])
print(ids_dtm[:2])
print(ids_slope[:2])
'''
print(ids_ground_truth[:2])

['/mnt/sda1/tiles_2018_wien/ortho/tile_ortho_10315.tif', '/mnt/sda1/tiles_2018_wien/ortho/tile_ortho_18532.tif']
['/mnt/sda1/tiles_2018_wien/ground_truth/tile_ground_truth_10315.tif', '/mnt/sda1/tiles_2018_wien/ground_truth/tile_ground_truth_18532.tif']


In [8]:
from osgeo import gdalconst

def tif2array(input_file, dtype=np.uint8):
    """
    read GeoTiff and convert to numpy.ndarray.
    Inputs:
        input_file (str) : the name of input GeoTiff file.
    return:
        image(np.array) : image for each bands
        dataset : for gdal's data drive.
    """
    dataset = gdal.Open(input_file, gdal.GA_ReadOnly)
    # Allocate our array using the first band's datatype
    image_datatype = dataset.GetRasterBand(1).DataType
    image = np.zeros((dataset.RasterYSize, dataset.RasterXSize, dataset.RasterCount),
                     dtype=dtype)
    
    # Loop over all bands in dataset
    for b in range(dataset.RasterCount):
        # Remember, GDAL index is on 1, but Python is on 0 -- so we add 1 for our GDAL calls
        band = dataset.GetRasterBand(b + 1)
        # Read in the band's data into the third dimension of our array
        image[:, :, b] = band.ReadAsArray()#buf_type=gdalconst.GDT_Byte)
        
    #image = image[2:-2,2:-2,:]
        
    return image

In [9]:
def cut_img(img, x, y):
    # set pixel sizes
    x_i, y_i, z_i = img.shape
    # dict to store the sliceing information
    d = {}
    
    for var, var_i, key in [(x, x_i, 'x'), (y, y_i, 'y')]:
        # if image pixel size is grater than the target pixel size
        if (var_i > var):
            # if even cut same amount of pixels from both sides
            if var_i%2 == 0:
                sub = int(var_i/2 - var/2)
                d[key+'0'] = sub
                d[key+'1'] = sub
            # if odd cut 1 pixel more from right/bottom
            else:
                sub = int(var_i/2 - var/2)
                d[key+'0'] = sub
                d[key+'1'] = sub + 1
        else:
            print('image too small')
    # cut image
    img = img[d['x0']:-d['x1'],d['y0']:-d['y1']]
    
    return img

In [10]:
# pass ids_ortho, ids_dsm, ids_dtm, ids_slope
def create_array(ids, dtype):
    
    imgs = []
    
    if dtype == np.uint8:
        # add all
        for i in ids:
            # load image to numpy array
            img = tif2array(i, np.uint8)
            # cut into right shape
            img = cut_img(img, 512, 512)
            # append array to list
            imgs.append(img)
            
        # convert list with arrays to numpy array
        arr = np.stack(imgs, axis=0)
        print(arr.shape)
            
        # calculate mean
        #mean = np.mean(arr, dtype='float32')
        # calculate standard deviation
        #std = np.std(arr, dtype='float32')
    
    
    
    
    else:
        # add all
        for i in ids:
            
            # load image to numpy array
            img = tif2array(i, np.float32)
            # cut into right shape
            img = cut_img(img, 512, 512)
            # append array to list
            imgs.append(img)
            x, y, z = img.shape

        # convert list with arrays to numpy array
        arr = np.stack(imgs, axis=0)
        arr[arr < 0] = np.nan
        print(arr.shape)

        # calculate mean
        #mean = np.nanmean(arr, dtype='float32')
        # calculate standard deviation
        #std = np.nanstd(arr, dtype='float32')
        # convert nan to 0
        arr = np.nan_to_num(arr)
    
    # normalize data
    #arr = (arr - mean) / std
    
    return arr#, mean, std

In [14]:
arr_ortho = create_array(ids_ortho, np.uint8)
arr_dsm = create_array(ids_dsm, np.float32)
arr_dtm = create_array(ids_dtm, np.float32)
arr_slope = create_array(ids_slope, np.float32)
arr_ground_truth = create_array(ids_ground_truth, np.uint8)

KeyboardInterrupt: 

In [25]:
# concatonate

In [26]:
data = []
for i in range(len(arr_ortho)):
    data.append(np.concatenate([arr_ortho[0], arr_dsm[0], arr_dtm[0], arr_slope[0]], axis=2))
# convert list with arrays to numpy array
data = np.stack(data, axis=0)

In [27]:
data.shape

(100, 512, 512, 7)

In [20]:
arr_ortho[0].shape

(100, 512, 512, 4)

In [21]:
arr_dsm

(array([[[[-1.1781157 ],
          [-1.1781157 ],
          [-1.1781157 ],
          ...,
          [ 0.7929953 ],
          [ 0.7929953 ],
          [ 0.7929953 ]],
 
         [[-1.1781157 ],
          [-1.1781157 ],
          [-1.1781157 ],
          ...,
          [ 0.7929953 ],
          [ 0.7929953 ],
          [ 0.7929953 ]],
 
         [[-1.1781157 ],
          [-1.1781157 ],
          [-1.1781157 ],
          ...,
          [ 0.7929953 ],
          [ 0.7929953 ],
          [ 0.7929953 ]],
 
         ...,
 
         [[-1.1781157 ],
          [-1.1781157 ],
          [-1.1781157 ],
          ...,
          [ 1.02835   ],
          [ 1.02835   ],
          [ 1.02835   ]],
 
         [[-1.1781157 ],
          [-1.1781157 ],
          [-1.1781157 ],
          ...,
          [ 1.02835   ],
          [ 1.02835   ],
          [ 1.02835   ]],
 
         [[-1.1781157 ],
          [-1.1781157 ],
          [-1.1781157 ],
          ...,
          [ 1.02835   ],
          [ 1.02835   ],
    

In [28]:
imgs_ortho = []
# add all 
for ids in ids_ortho:
    # load image to numpy array
    img_ortho = tif2array(ids, np.uint8)
    # cut into right shape
    img_ortho = img_ortho[1:-2,1:-2]
    # append array to list
    imgs_ortho.append(img_ortho)

# convert list with arrays to numpy array
arr_ortho = np.stack(imgs_ortho, axis=0)
print(arr_ortho.shape)

# calculate mean
mean_ortho = np.mean(arr_ortho, dtype='float32')
# calculate standard deviation
std_ortho = np.std(arr_ortho, dtype='float32')

# normalize data
arr_ortho = (arr_ortho - mean_ortho) / std_ortho

(100, 512, 512, 4)


In [29]:
arr_ortho

array([[[[-1.1282684 , -1.0664089 , -1.1076485 , -0.6746317 ],
         [-1.1076485 , -1.0251691 , -1.045789  , -0.6746317 ],
         [-1.1282684 , -1.045789  , -1.0664089 , -0.77773094],
         ...,
         [-1.1695081 , -1.1282684 , -1.1488882 , -0.5715325 ],
         [-1.0664089 , -1.0251691 , -1.0251691 , -0.26223478],
         [-1.0664089 , -1.0251691 , -1.0251691 , -0.17975539]],

        [[-1.1488882 , -1.0870287 , -1.1282684 , -0.77773094],
         [-1.1695081 , -1.0870287 , -1.1282684 , -0.8395905 ],
         [-1.1488882 , -1.0664089 , -1.1076485 , -0.86021036],
         ...,
         [-1.1076485 , -1.0251691 , -1.0251691 , -0.4890531 ],
         [-1.0664089 , -1.0251691 , -1.0251691 , -0.38595387],
         [-1.0870287 , -1.045789  , -1.045789  , -0.365334  ]],

        [[-1.1488882 , -1.0664089 , -1.1076485 , -0.77773094],
         [-1.1695081 , -1.0664089 , -1.1076485 , -0.8395905 ],
         [-1.1695081 , -1.0870287 , -1.1282684 , -0.90145004],
         ...,
         

In [12]:
# save data in hdf5 file

In [47]:
# initialising hdf5
ds_file_name  = "/home/philipp/Data/edin_dataset/dataset_512_1.h5"

ortho_shape = (len(arr_ortho), 512, 512, 4)
dsm_shape = (len(arr_dsm), 512, 512, 1)

with h5py.File(ds_file_name, 'w') as hf:
    hf.create_dataset('x_ortho', data=arr_ortho, shape=ortho_shape, dtype=np.uint8, chunks=(25,512,512,4))
    hf.create_dataset('x_dsm', data=arr_dsm, shape=dsm_shape, dtype=np.float, chunks=(25,512,512,1))
    hf.create_dataset('x_dtm', data=arr_dtm, shape=dsm_shape, dtype=np.float32, chunks=(25,512,512,1))
    hf.create_dataset('x_slope', data=arr_slope, shape=dsm_shape, dtype=np.float32, chunks=(25,512,512,1))
    hf.create_dataset('y_ground_truth', data=arr_ground_truth, shape=dsm_shape, dtype=np.uint8, chunks=(25,512,512,1))

In [31]:
# read HDF5 file
with h5py.File("/home/philipp/Data/edin_dataset/dataset_512_1.h5", 'r') as hf:
    dset_x_train = np.array(hf['x_ortho'])
    dset_y_train = np.array(hf['x_dsm'])
    dset_x_test = np.array(hf['x_dtm'])
    dset_y_test = np.array(hf['x_slope'])
    
print(dset_x_train.shape)
print(dset_y_train.shape)
print(dset_x_test.shape)
print(dset_y_test.shape)

(100, 512, 512, 4)
(100, 512, 512, 1)
(100, 512, 512, 1)
(100, 512, 512, 1)


In [75]:
dset = h5py.File("/home/philipp/Data/edin_dataset/dataset_512_1.h5", 'r')

In [78]:
dset.keys()

<KeysViewHDF5 ['x_dsm', 'x_dtm', 'x_ortho', 'x_slope', 'y_ground_truth']>

In [79]:
x_dsm = dset['x_dsm']

In [85]:
x_dsm.shape[0]

100

In [ ]:
dset.close()

In [162]:
dset_x_train

array([[[[-1.1282684 , -1.0664089 , -1.1076485 , -0.6746317 ],
         [-1.1076485 , -1.0251691 , -1.045789  , -0.6746317 ],
         [-1.1282684 , -1.045789  , -1.0664089 , -0.77773094],
         ...,
         [-1.1695081 , -1.1282684 , -1.1488882 , -0.5715325 ],
         [-1.0664089 , -1.0251691 , -1.0251691 , -0.26223478],
         [-1.0664089 , -1.0251691 , -1.0251691 , -0.17975539]],

        [[-1.1488882 , -1.0870287 , -1.1282684 , -0.77773094],
         [-1.1695081 , -1.0870287 , -1.1282684 , -0.8395905 ],
         [-1.1488882 , -1.0664089 , -1.1076485 , -0.86021036],
         ...,
         [-1.1076485 , -1.0251691 , -1.0251691 , -0.4890531 ],
         [-1.0664089 , -1.0251691 , -1.0251691 , -0.38595387],
         [-1.0870287 , -1.045789  , -1.045789  , -0.365334  ]],

        [[-1.1488882 , -1.0664089 , -1.1076485 , -0.77773094],
         [-1.1695081 , -1.0664089 , -1.1076485 , -0.8395905 ],
         [-1.1695081 , -1.0870287 , -1.1282684 , -0.90145004],
         ...,
         

In [ ]:
# possibility of saving arrays bigger than 

In [49]:
hdf5_store = h5py.File("/home/philipp/Data/edin_dataset/dataset_test.h5", 'a')

In [53]:
hdf5_store.keys()

<KeysViewHDF5 ['x_ortho']>

In [55]:
x = hdf5_store['x_ortho']

In [58]:
x

<HDF5 dataset "x_ortho": shape (200, 512, 512, 4), type "<f4">

In [61]:
x[10].shape

(512, 512, 4)

In [69]:
x[100:150,:,:,:] = arr_ortho[:50]

In [73]:
x[100]

array([[[ 40.,  43.,  41.,  62.],
        [ 41.,  45.,  44.,  62.],
        [ 40.,  44.,  43.,  57.],
        ...,
        [ 38.,  40.,  39.,  67.],
        [ 43.,  45.,  45.,  82.],
        [ 43.,  45.,  45.,  86.]],

       [[ 39.,  42.,  40.,  57.],
        [ 38.,  42.,  40.,  54.],
        [ 39.,  43.,  41.,  53.],
        ...,
        [ 41.,  45.,  45.,  71.],
        [ 43.,  45.,  45.,  76.],
        [ 42.,  44.,  44.,  77.]],

       [[ 39.,  43.,  41.,  57.],
        [ 38.,  43.,  41.,  54.],
        [ 38.,  42.,  40.,  51.],
        ...,
        [ 42.,  44.,  45.,  67.],
        [ 41.,  43.,  43.,  66.],
        [ 43.,  45.,  45.,  74.]],

       ...,

       [[ 45.,  46.,  43.,  69.],
        [ 44.,  46.,  43.,  68.],
        [ 48.,  50.,  48.,  74.],
        ...,
        [ 46.,  47.,  42., 126.],
        [ 51.,  52.,  45., 136.],
        [ 47.,  48.,  41., 131.]],

       [[ 45.,  47.,  45.,  70.],
        [ 46.,  48.,  46.,  70.],
        [ 46.,  47.,  45.,  68.],
        .

In [64]:
arr_ortho[:50,:,:,:]

array([[[[ 40,  43,  41,  62],
         [ 41,  45,  44,  62],
         [ 40,  44,  43,  57],
         ...,
         [ 38,  40,  39,  67],
         [ 43,  45,  45,  82],
         [ 43,  45,  45,  86]],

        [[ 39,  42,  40,  57],
         [ 38,  42,  40,  54],
         [ 39,  43,  41,  53],
         ...,
         [ 41,  45,  45,  71],
         [ 43,  45,  45,  76],
         [ 42,  44,  44,  77]],

        [[ 39,  43,  41,  57],
         [ 38,  43,  41,  54],
         [ 38,  42,  40,  51],
         ...,
         [ 42,  44,  45,  67],
         [ 41,  43,  43,  66],
         [ 43,  45,  45,  74]],

        ...,

        [[ 45,  46,  43,  69],
         [ 44,  46,  43,  68],
         [ 48,  50,  48,  74],
         ...,
         [ 46,  47,  42, 126],
         [ 51,  52,  45, 136],
         [ 47,  48,  41, 131]],

        [[ 45,  47,  45,  70],
         [ 46,  48,  46,  70],
         [ 46,  47,  45,  68],
         ...,
         [ 54,  55,  49, 146],
         [ 49,  49,  43, 137],
         

In [48]:
hdf5_store = h5py.File("/home/philipp/Data/edin_dataset/dataset_test.h5", 'a')
# seting space to shape 200,512,512,4
x_ortho = hdf5_store.create_dataset("x_ortho", (200,512,512,4), dtype=np.float32, chunks=True)

RuntimeError: Unable to create link (name already exists)

In [166]:
# assigning "arr_ortho" to the first 100 samples
x_ortho[:100,:,:,:] = arr_ortho

In [74]:
hdf5_store.close()

In [168]:
with h5py.File("/home/philipp/Data/edin_dataset/dataset_test.h5", 'r') as hf:
    dset_x_train = np.array(hf['x_ortho'])

In [170]:
dset_x_train.shape

(200, 512, 512, 4)

In [172]:
dset_x_train[100]

array([[[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        ...,
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]],

       [[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        ...,
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]],

       [[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        ...,
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]],

       ...,

       [[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        ...,
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]],

       [[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        ...,
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]],

       [[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        ...,
        [0., 0., 0., 0.],
        [0., 0.